In [1]:
import pandas as pd

quick = True

if quick:
    start_year, end_year, chk, sampled = [2005, 2021, False, True]
else:
    start_year, end_year, chk, sampled = [2005, 2021, True, False]

undersampling = True
oversampling  = False
smote         = False

In [2]:
from sklearn.model_selection import train_test_split
from my_libs.scalers import FeaturesScaler

df = pd.read_pickle("./project_df.pkl")
data = df.iloc[:, 1:]
target = df['grav']

X_train, X_test, y_train, y_test = train_test_split(data, target, test_size=0.2, random_state=222)

In [3]:
# from my_libs.encoders import FeaturesEncoder
# from sklearn.tree import DecisionTreeClassifier
# from imblearn.over_sampling import SMOTE
# from imblearn.metrics import classification_report_imbalanced
#
# encoder = FeaturesEncoder(True)
# scaler  = FeaturesScaler(True)
# smt     = SMOTE(random_state=42)
# model   = DecisionTreeClassifier(max_depth=10)
#
# X_train = encoder.fit_transform(X_train, y_train)
# X_train = scaler.fit_transform(X_train, y_train)
# X_train, y_train = smt.fit_resample(X_train, y_train)
# model.fit(X_train, y_train)
#
# X_test = encoder.fit_transform(X_test, y_test)
# X_test = scaler.fit_transform(X_test, y_test)
# X_test, y_test = smt.fit_resample(X_test, y_test)
# y_pred = model.predict(X_test)
# print(classification_report_imbalanced(y_test, y_pred))

In [10]:
from my_libs.encoders import FeaturesEncoder
from sklearn.tree import DecisionTreeClassifier
# from sklearn.pipeline import Pipeline
from imblearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV
from imblearn.over_sampling import SMOTE
# from sklearn.metrics import classification_report
from imblearn.metrics import classification_report_imbalanced

encoder = FeaturesEncoder(False)
scaler  = FeaturesScaler()
smt     = SMOTE(random_state=42)
model   = DecisionTreeClassifier()

dt_pipe = Pipeline([
                    ('var_encoding', encoder),
                    ('scaling', scaler),
                    ('smote', smt),
                    # ('selection', selector),
                    ('model', model)
                    ])
param_grid = {
    'model__max_depth' : (10, 20),
    # 'model__criterion' : ('gini', 'entropy'),
    # 'model__max_features' : ('auto', 'sqrt', 'log2'),
    # 'model__min_samples_split' : (2,4,6)
}
print(X_train.shape)
dt_pipe.fit(X_train, y_train)
#
# grid = GridSearchCV(dt_pipe, param_grid=param_grid, cv = 3)
# grid.fit(X_train, y_train)
#
# print('Le meilleur score obtenu est: ',grid.best_score_)
# print('Les meilleurs paramètres trouvés sont :',grid.best_params_)
print(X_test.shape)
y_pred = dt_pipe.predict(X_test)
print(y_pred.shape)

print(classification_report_imbalanced(y_test, y_pred))


(79832, 28)
(19958, 28)


ValueError: Lengths must match to compare

In [ ]:
# y_pred = dt_pipe.predict(X_test)
# print(classification_report_imbalanced(y_test, y_pred))

In [5]:
X_train.head()

,place,catu,sexe,trajet,locp,actp,etatp,mois,jour,lum,...,vosp,prof,plan,surf,infra,situ,senc,catv,age,joursem
641454,1,1,2,1,0,0,0,6,0.224923,1,...,0,1,4,1,0,1,0,7,0.249003,mardi
2275041,1,1,1,0,0,0,-1,3,0.232585,1,...,0,1,1,1,0,1,1,7,0.214706,samedi
1167782,1,1,1,5,0,0,0,11,0.232342,1,...,0,1,1,1,0,1,0,7,0.240860,dimanche
1730875,1,1,1,5,0,0,0,7,0.230705,1,...,0,1,1,1,0,1,0,10,0.219430,jeudi
1668029,1,1,2,2,0,0,0,12,0.226168,2,...,0,1,1,2,0,1,0,1,0.332657,mardi


In [ ]:
# from imblearn.under_sampling import RandomUnderSampler
# from imblearn.over_sampling import RandomOverSampler, SMOTE
#
# if undersampling:
#     # Random Undersampling
#     rUs = RandomUnderSampler()
#     # X_ru, y_ru = rUs.fit_resample(X_train, y_train)
#     X_train, y_train = rUs.fit_resample(X_train, y_train)
#     # print('Classes échantillon undersampled :', y_ru.value_counts())
#     print('Classes échantillon undersampled :', y_train.value_counts())
# if oversampling:
#     rOs = RandomOverSampler()
#     X_train, y_train = rOs.fit_resample(X_train, y_train)
#     print('Classes échantillon oversampled :', dict(pd.Series(y_train).value_counts()))
# if smote:
#     smo = SMOTE()
#     X_train, y_train = smo.fit_resample(X_train, y_train)
#     print('Classes échantillon SMOTE :', dict(pd.Series(y_train).value_counts()))

In [ ]:
# from sklearn.model_selection import RandomizedSearchCV
# from sklearn.tree import DecisionTreeClassifier
#
# parameters = {'max_depth'         : (10,20,30,40,50),
#               'criterion'         : ('gini', 'entropy'),
#               'max_features'      : ('auto', 'sqrt', 'log2'),
#               'min_samples_split' : (2,4,6)
#               }
#
# grid = RandomizedSearchCV(DecisionTreeClassifier(), param_distributions=parameters, cv=5, verbose=True)
#
# grid.fit(X_train, y_train)

In [ ]:
# from imblearn.metrics import classification_report_imbalanced
#
# model = grid.best_estimator_
# print(model)
#
# model.fit(X_train, y_train)
# y_pred = model.predict(X_test)
# print(pd.crosstab(y_test, y_pred, rownames=['Classe réelle'], colnames=['Classe prédite']))
#
# print(f"\n{classification_report_imbalanced(y_test, y_pred)}")


In [ ]:
# feats = {}
# for feature, importance in zip(data.columns, model.feature_importances_):
#     feats[feature] = importance
#
# importances = pd.DataFrame.from_dict(feats, orient='index').rename(columns={0: 'Gini-importance'})
#
# #Affichage des 8 variables les plus importantes
# importances.sort_values(by='Gini-importance', ascending=False).head(8)

In [ ]:
# from sklearn import tree
# tree.plot_tree(model);


In [ ]:
# print(f"Train accuracy : {model.score(X_train, y_train):.3f}")
# print(f"Test accuracy : {model.score(X_test, y_test):.3f}")